In [1]:
# Step 0. Load libraries and custom functions
# Matrices and datasets ------------------------------------------------
import pandas as pd
import numpy as np
# Graphics -------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Text processors
import re
import string
#import nltk
#from nltk.corpus import stopwords
#nltk.download('stopwords')
from wordcloud import WordCloud
# Machine Learning -----------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
# Deep Learning --------------------------------------------------------
import keras
import tensorflow as tf
from keras import layers
from keras.layers import TextVectorization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
# Step 1. Load data
# 1.1 Read csv and get basic info
df_raw = pd.read_csv('../data/02_TMDB_5000_movies.csv')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [3]:
# 1.2 Get a sample
df_raw.sample(10, random_state=2024)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2182,0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",NaN,13682,[],en,Pooh's Heffalump Movie,Who or what exactly is a Heffalump? The lovabl...,9.031540,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-02-11,0,68.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's something new in the Hundred Acre Wood.,Pooh's Heffalump Movie,6.4,88
3274,8000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",NaN,13154,"[{""id"": 1794, ""name"": ""yakuza""}, {""id"": 12670,...",en,Showdown in Little Tokyo,"An American with a Japanese upbringing, Chris ...",8.403859,"[{""name"": ""Original Pictures"", ""id"": 4234}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-08-23,2275557,79.0,"[{""iso_639_1"": ""ja"", ""name"": ""\u65e5\u672c\u8a...",Released,One's a warrior. One's a wise guy. They're two...,Showdown in Little Tokyo,5.7,95
1003,49000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",NaN,9548,"[{""id"": 578, ""name"": ""rock and roll""}, {""id"": ...",en,The Adventures of Ford Fairlane,"Ford ""Mr. Rock n' Roll Detective"" Fairlane is ...",2.808428,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1990-07-11,20423389,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Kojak. Columbo. Dirty Harry. Wimps.,The Adventures of Ford Fairlane,6.2,71
1383,32000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,13920,"[{""id"": 5565, ""name"": ""biography""}, {""id"": 605...",en,Radio,"High school football coach, Harold Jones befri...",9.254647,"[{""name"": ""Revolution Studios"", ""id"": 497}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2003-10-24,52277485,109.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,His courage made them champions.,Radio,6.8,141
2724,18339750,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",http://www.downfallthefilm.com/,613,"[{""id"": 220, ""name"": ""berlin""}, {""id"": 351, ""n...",de,Der Untergang,"In April of 1945, Germany stands at the brink ...",32.445895,"[{""name"": ""Degeto Film"", ""id"": 986}, {""name"": ...","[{""iso_3166_1"": ""AT"", ""name"": ""Austria""}, {""is...",2004-09-08,92180910,156.0,"[{""iso_639_1"": ""hu"", ""name"": ""Magyar""}, {""iso_...",Released,"April 1945, a nation awaits its...Downfall",Downfall,7.7,1037
3340,7000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,713,"[{""id"": 128, ""name"": ""love triangle""}, {""id"": ...",en,The Piano,"After a long voyage from Scotland, pianist Ada...",17.681707,"[{""name"": ""New South Wales Film & Television O...","[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}, ...",1993-05-19,116700000,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,NaN,The Piano,7.1,281
463,0,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 18, ...",NaN,161795,"[{""id"": 9673, ""name"": ""love""}, {""id"": 14638, ""...",en,Déjà Vu,L.A. shop owner Dana and Englishman Sean meet ...,0.605645,"[{""name"": ""Rainbow Film Company, The"", ""id"": 2...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1998-04-22,0,117.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Your future is set...,Déjà Vu,8.0,1
4168,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",NaN,356987,"[{""id"": 230912, ""name"": ""supervivencia""}]",en,Abandoned,When their yacht capsizes during a storm; four...,3.068463,"[{""name"": ""Making Movies"", ""id"": 71702}]","[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}]",2015-08-30,0,82.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Abandoned,5.8,27
4057,2160000,"[{""id"": 18, ""name

Some of the columns contains nested json data, other contains unique 
information like ids or names, so let's transform our dataset. 

In [32]:
# 2. Preprocess data
# 2.1 Create a interim dataset for transformations, drop unused columns
df_interim = df_raw.copy()
df_interim = df_interim.drop(columns=['id','original_title','title','vote_count','original_language','homepage'])

Now we can concatenate each id value in json format to form a id collection
and process it for prediction

In [33]:
# 2.2 Concatenate strings in json format and drop changed columns
df_interim['genres_c'] = df_interim['genres'].apply(lambda x: ' '.join([str(y['id']) for y in eval(x)]))
df_interim['keywords_c'] = df_interim['keywords'].apply(lambda x: ' '.join([str(y['id']) for y in eval(x)]))
df_interim['producers_c'] = df_interim['production_companies'].apply(lambda x: ' '.join([str(y['id']) for y in eval(x)]))
df_interim['countries_c'] = df_interim['production_countries'].apply(lambda x: ' '.join([str(y['iso_3166_1']) for y in eval(x)]))
df_interim['languages_c'] = df_interim['spoken_languages'].apply(lambda x: ' '.join([str(y['iso_639_1']) for y in eval(x)]))
df_interim = df_interim.drop(columns=['genres','keywords','production_companies','production_countries','spoken_languages'])

In [34]:
# 2.3 View results and current shape
display(df_interim.sample(2))
df_interim.shape

,budget,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,genres_c,keywords_c,producers_c,countries_c,languages_c
3460,6000000,"In 1997, the island of Manhattan has been wall...",24.351305,1981-05-22,50244700,99.0,Released,1997. New York City is now a maximum security ...,6.9,878 28,444 542 840 1543 1562 1930 2019 2095 3149 4565...,494 500 502 664,US,en
762,60000000,Renegade FBI agent Art Jeffries protects a nin...,18.877795,1998-04-03,0,111.0,Released,Someone knows too much.,6.0,28 80 18 53,782 1465 1646 1812 3432 3635 5202 10656 18525 ...,23 33,US,en


(4803, 14)

Now some information comes as numeric like budget. But since values in 
budgets are quite large, we can apply some transformations like log. To 
avoid zeros, we can add 1 to all values.

In [35]:
# 2.4 Transform scale in numeric variables
df_interim['budget_log'] = np.log(df_interim['budget']+1)
df_interim['revenue_log'] = np.log(df_interim['revenue']+1)

About the date, we can use a point of reference, like the year of the 
oldest movie as starting point. 

In [39]:
df_interim['Month_t'] = df_interim['release_date'].apply(lambda x: str(x)[5:7])

In [45]:
df_interim.loc[df_interim.Month_t == '']

,budget,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,genres_c,keywords_c,producers_c,countries_c,languages_c,budget_log,revenue_log,Year_t,Month_t
4553,0,1971 post civil rights San Francisco seemed li...,0.0,NaN,0,0.0,Released,NaN,0.0,,,,,,0.0,0.0,NaN,


In [44]:
df_interim['Month_t'].apply(lambda x: float(x) if x!='' else 1)

0       12.0
1        5.0
2       10.0
3        7.0
4        3.0
        ... 
4798     9.0
4799    12.0
4800    10.0
4801     5.0
4802     8.0
Name: Month_t, Length: 4803, dtype: float64

In [43]:
df_interim['Month_t'].value_counts(dropna=False)

Month_t
09    588
10    474
12    456
08    413
06    389
01    382
03    372
05    363
07    363
04    350
11    326
02    326
        1
Name: count, dtype: int64

In [38]:
# 2.5 Transform the date
df_interim['Year_t'] = df_interim['release_date'].apply(lambda x: float(str(x)[0:4]))
df_interim['Month_t'] = df_interim['release_date'].apply(lambda x: int(str(x)[5:7]))
df_interim = df_interim.drop(columns=['release_date'])
df_interim['Year_diff'] = df_interim['Year_t'] - min(df_interim['Year_t'])

ValueError: invalid literal for int() with base 10: ''

In [20]:
# 2.5 Transform the date
# df_interim = df_interim.join(
#     df_interim[['release_date']].apply(
#         func = (lambda x: [int(x['release_date'][:4]),int(x['release_date'][5:7])]),
#         result_type='expand',
#         axis=1
#     )
# )
df_interim = df_interim.join(df_interim[['release_date']].apply(
    axis=1, result_type='expand',
    func=(lambda x : [int(x['release_date'][:4]), int(x['release_date'][5:7])])))
#df_interim = df_interim.rename(columns={0:'Year',1:'Month'})

TypeError: 'float' object is not subscriptable

### References
[1] https://github.com/PhilChodrow/PIC16B/blob/7d12d32e070e7ff3840b971c0ce4185ef1911796/discussion/tmdb.ipynb#L758